In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
#stop_words = stopwords.words('english')

#from text_share.text_share3_5noisyexamples import noisy_example1, noisy_example2, noisy_example3, noisy_example1_morph, noisy_example2_morph

In [3]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

def prediction_accuracy(predictions, yvalid):
    return len([i for i, j in zip(predictions.argmax(axis=1), yvalid) if i == j])/len(yvalid)

In [5]:
#lbl_enc = preprocessing.LabelEncoder()
#ytrain = lbl_enc.fit_transform(edit_data.qti.values)

In [9]:
cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='char_wb',ngram_range=(1,4),
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne0'].values.tolist())
train_cvec_char0 = cvec_char_vec.transform(raw_data['edit_text_ne0'].to_numpy())
test_cvec_char0 = cvec_char_vec.transform(raw_test_data['edit_text_ne0'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='char_wb',ngram_range=(1,4),
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne1'].values.tolist())
train_cvec_char1 = cvec_char_vec.transform(raw_data['edit_text_ne1'].to_numpy())
test_cvec_char1 = cvec_char_vec.transform(raw_test_data['edit_text_ne1'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='char_wb',ngram_range=(1,4),
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne2'].values.tolist())
train_cvec_char2 = cvec_char_vec.transform(raw_data['edit_text_ne2'].to_numpy())
test_cvec_char2 = cvec_char_vec.transform(raw_test_data['edit_text_ne2'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='char_wb',ngram_range=(1,4),
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne3'].values.tolist())
train_cvec_char3 = cvec_char_vec.transform(raw_data['edit_text_ne3'].to_numpy())
test_cvec_char3 = cvec_char_vec.transform(raw_test_data['edit_text_ne3'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='char_wb',ngram_range=(1,4),
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne4'].values.tolist())
train_cvec_char4 = cvec_char_vec.transform(raw_data['edit_text_ne4'].to_numpy())
test_cvec_char4 = cvec_char_vec.transform(raw_test_data['edit_text_ne4'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='char_wb',ngram_range=(1,4),
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne5'].values.tolist())
train_cvec_char5 = cvec_char_vec.transform(raw_data['edit_text_ne5'].to_numpy())
test_cvec_char5 = cvec_char_vec.transform(raw_test_data['edit_text_ne5'].to_numpy())

In [10]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char0, raw_data['label'])
test_predictions0 = clf.predict_proba(test_cvec_char0.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char1, raw_data['label'])
test_predictions1 = clf.predict_proba(test_cvec_char1.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char2, raw_data['label'])
test_predictions2 = clf.predict_proba(test_cvec_char2.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char3, raw_data['label'])
test_predictions3 = clf.predict_proba(test_cvec_char3.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char4, raw_data['label'])
test_predictions4 = clf.predict_proba(test_cvec_char4.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char5, raw_data['label'])
test_predictions5 = clf.predict_proba(test_cvec_char5.tocsc())

In [ ]:
################### SAVE FIRST #########################################################

In [ ]:
#################################down here for xgb 3#######################################

In [11]:
cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='word',
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne0m'].values.tolist())
train_cvec_char0 = cvec_char_vec.transform(raw_data['edit_text_ne0m'].to_numpy())
test_cvec_char0 = cvec_char_vec.transform(raw_test_data['edit_text_ne0m'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='word',
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne1m'].values.tolist())
train_cvec_char1 = cvec_char_vec.transform(raw_data['edit_text_ne1m'].to_numpy())
test_cvec_char1 = cvec_char_vec.transform(raw_test_data['edit_text_ne1m'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='word',
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne2m'].values.tolist())
train_cvec_char2 = cvec_char_vec.transform(raw_data['edit_text_ne2m'].to_numpy())
test_cvec_char2 = cvec_char_vec.transform(raw_test_data['edit_text_ne2m'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='word',
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne3m'].values.tolist())
train_cvec_char3 = cvec_char_vec.transform(raw_data['edit_text_ne3m'].to_numpy())
test_cvec_char3 = cvec_char_vec.transform(raw_test_data['edit_text_ne3m'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='word',
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne4m'].values.tolist())
train_cvec_char4 = cvec_char_vec.transform(raw_data['edit_text_ne4m'].to_numpy())
test_cvec_char4 = cvec_char_vec.transform(raw_test_data['edit_text_ne4m'].to_numpy())

cvec_char_vec = CountVectorizer(min_df=3,  max_features=None, 
                                strip_accents='unicode', analyzer='word',
                                stop_words = None)

#cvec_char_vec=CountVectorizer(min_df=3,tokenizer=tokenizer.tokenize,ngram_range=(1,5))
cvec_char_vec.fit(raw_data['edit_text_ne5m'].values.tolist())
train_cvec_char5 = cvec_char_vec.transform(raw_data['edit_text_ne5m'].to_numpy())
test_cvec_char5 = cvec_char_vec.transform(raw_test_data['edit_text_ne5m'].to_numpy())

In [12]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char0, raw_data['label'])
test_predictions0m = clf.predict_proba(test_cvec_char0.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char1, raw_data['label'])
test_predictions1m = clf.predict_proba(test_cvec_char1.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char2, raw_data['label'])
test_predictions2m = clf.predict_proba(test_cvec_char2.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char3, raw_data['label'])
test_predictions3m = clf.predict_proba(test_cvec_char3.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char4, raw_data['label'])
test_predictions4m = clf.predict_proba(test_cvec_char4.tocsc())

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(train_cvec_char5, raw_data['label'])
test_predictions5m = clf.predict_proba(test_cvec_char5.tocsc())

In [13]:
FILEPATH = '../Mondeique/predictions_clean/'

np.save(FILEPATH+'XGB_'+'ne0',test_predictions0)
np.save(FILEPATH+'XGB_'+'ne1',test_predictions1)
np.save(FILEPATH+'XGB_'+'ne2',test_predictions2)
np.save(FILEPATH+'XGB_'+'ne3',test_predictions3)
np.save(FILEPATH+'XGB_'+'ne4',test_predictions4)
np.save(FILEPATH+'XGB_'+'ne5',test_predictions5)

np.save(FILEPATH+'XGB_'+'ne0m',test_predictions0m)
np.save(FILEPATH+'XGB_'+'ne1m',test_predictions1m)
np.save(FILEPATH+'XGB_'+'ne2m',test_predictions2m)
np.save(FILEPATH+'XGB_'+'ne3m',test_predictions3m)
np.save(FILEPATH+'XGB_'+'ne4m',test_predictions4m)
np.save(FILEPATH+'XGB_'+'ne5m',test_predictions5m)

In [401]:
open_train_data['edit_text_ne5']

0        아래쪽 그림과 같이 한 변의 길이가 인 정사각형에서 각 변의 중점 , 에서 반지름...
1                                    함수 의 역함수를 바르게 구한 것은 
2                      함수 의 최댓값을 최솟값을 라고 할 때 값은 단 는 상수이다 
3                                     다음 명제 중 그 역이 참인 것은 
4        어느 고등학교 학생 명의 달리기 기록은 평균이 분 표준편차가 분인 정규분포를 따른...
                              ...                        
2677                         두 사건 에 대하여 br/> , 일 때 br/ 는 
2678          무리함수 의 그래프가 다음 그림과 같을 때 상수 에 대하여 의 값을 구하시오 
2679      함수 의 역함수를 라고 하자 의 그래프와 의 그래프의 교점의 좌표가 일 때 의 값은 
2680                             함수 에 대하여 를 만족하는 실수 의 값은 
2681     정규분포 을 따르는 모집단에서 크기가 인 표본을 임의추출할 때 표본평균 에 대하여...
Name: edit_text_ne5, Length: 2682, dtype: object

In [402]:
zip_list = zip(open_test_data['label'].values, test_predictions0.argmax(axis=1))
correct = 0
count = 0
wrong0 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0.append(count)
    count+=1
percentage0 = correct/len(open_test_data['label'])

In [403]:
zip_list = zip(open_test_data['label'].values, test_predictions1.argmax(axis=1))
correct = 0
count = 0
wrong1 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1.append(count)
    count+=1
percentage1 = correct/len(open_test_data['label'])

In [404]:
zip_list = zip(open_test_data['label'].values, test_predictions2.argmax(axis=1))
correct = 0
count = 0
wrong2 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2.append(count)
    count+=1
percentage2 = correct/len(open_test_data['label'])

In [405]:
zip_list = zip(open_test_data['label'].values, test_predictions3.argmax(axis=1))
correct = 0
count = 0
wrong3 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3.append(count)
    count+=1
percentage3 = correct/len(open_test_data['label'])

In [406]:
zip_list = zip(open_test_data['label'].values, test_predictions4.argmax(axis=1))
correct = 0
count = 0
wrong4 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4.append(count)
    count+=1
percentage4 = correct/len(open_test_data['label'])

In [407]:
zip_list = zip(open_test_data['label'].values, test_predictions5.argmax(axis=1))
correct = 0
count = 0
wrong5 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5.append(count)
    count+=1
percentage5 = correct/len(open_test_data['label'])

In [408]:
print(percentage0, percentage1, percentage2, percentage3, percentage4, percentage5)

0.8266219239373602 0.831096196868009 0.825503355704698 0.8277404921700223 0.6976137211036539 0.6991051454138703


In [409]:
print(len(wrong0), wrong0)
print(len(wrong1),wrong1)
print(len(wrong2), wrong2)
print(len(wrong3),wrong3)
print(len(wrong4),wrong4)
print(len(wrong5),wrong5)

465 [0, 2, 7, 14, 19, 20, 25, 28, 33, 42, 44, 54, 57, 59, 69, 71, 74, 78, 81, 93, 98, 102, 104, 108, 122, 123, 130, 137, 142, 144, 147, 148, 149, 151, 157, 158, 172, 173, 179, 184, 186, 195, 199, 201, 203, 229, 230, 231, 236, 237, 252, 257, 259, 262, 263, 265, 276, 281, 284, 287, 288, 295, 297, 315, 325, 339, 341, 342, 362, 379, 393, 395, 396, 404, 405, 407, 426, 427, 432, 447, 454, 474, 477, 479, 497, 506, 511, 523, 531, 532, 534, 546, 556, 558, 561, 563, 570, 587, 591, 599, 602, 606, 615, 618, 656, 666, 668, 670, 672, 675, 677, 680, 685, 686, 693, 698, 701, 705, 707, 711, 723, 737, 742, 744, 746, 755, 759, 770, 771, 778, 787, 803, 810, 812, 821, 822, 826, 829, 831, 833, 839, 858, 861, 869, 875, 876, 883, 886, 894, 907, 913, 919, 920, 931, 933, 945, 953, 956, 958, 963, 967, 968, 985, 993, 997, 1001, 1008, 1009, 1011, 1013, 1020, 1022, 1025, 1030, 1031, 1033, 1034, 1043, 1050, 1057, 1064, 1077, 1080, 1092, 1093, 1097, 1114, 1119, 1125, 1126, 1128, 1138, 1139, 1142, 1143, 1147, 1151, 11

In [410]:
test_predictions0_1 = clf.predict_proba(test_cvec_char1.tocsc())
test_predictions0_2 = clf.predict_proba(test_cvec_char2.tocsc())
test_predictions0_3 = clf.predict_proba(test_cvec_char3.tocsc())
test_predictions0_4 = clf.predict_proba(test_cvec_char4.tocsc())
test_predictions0_5 = clf.predict_proba(test_cvec_char5.tocsc())

In [411]:
test_predictions1_0 = clf.predict_proba(test_cvec_char0.tocsc())
test_predictions1_2 = clf.predict_proba(test_cvec_char2.tocsc())
test_predictions1_3 = clf.predict_proba(test_cvec_char3.tocsc())
test_predictions1_4 = clf.predict_proba(test_cvec_char4.tocsc())
test_predictions1_5 = clf.predict_proba(test_cvec_char5.tocsc())

In [412]:
test_predictions2_0 = clf.predict_proba(test_cvec_char0.tocsc())
test_predictions2_1 = clf.predict_proba(test_cvec_char1.tocsc())
test_predictions2_3 = clf.predict_proba(test_cvec_char3.tocsc())
test_predictions2_4 = clf.predict_proba(test_cvec_char4.tocsc())
test_predictions2_5 = clf.predict_proba(test_cvec_char5.tocsc())

In [413]:
test_predictions3_1 = clf.predict_proba(test_cvec_char1.tocsc())
test_predictions3_2 = clf.predict_proba(test_cvec_char2.tocsc())
test_predictions3_0 = clf.predict_proba(test_cvec_char0.tocsc())
test_predictions3_4 = clf.predict_proba(test_cvec_char4.tocsc())
test_predictions3_5 = clf.predict_proba(test_cvec_char5.tocsc())

In [414]:
test_predictions4_1 = clf.predict_proba(test_cvec_char1.tocsc())
test_predictions4_2 = clf.predict_proba(test_cvec_char2.tocsc())
test_predictions4_3 = clf.predict_proba(test_cvec_char3.tocsc())
test_predictions4_0 = clf.predict_proba(test_cvec_char0.tocsc())
test_predictions4_5 = clf.predict_proba(test_cvec_char5.tocsc())

In [415]:
test_predictions5_1 = clf.predict_proba(test_cvec_char1.tocsc())
test_predictions5_2 = clf.predict_proba(test_cvec_char2.tocsc())
test_predictions5_3 = clf.predict_proba(test_cvec_char3.tocsc())
test_predictions5_4 = clf.predict_proba(test_cvec_char4.tocsc())
test_predictions5_0 = clf.predict_proba(test_cvec_char0.tocsc())

In [416]:
zip_list = zip(open_test_data['label'].values, test_predictions0_1.argmax(axis=1))
correct = 0
count = 0
wrong0_1 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_1.append(count)
    count+=1
percentage0_1 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions0_2.argmax(axis=1))
correct = 0
count = 0
wrong0_2 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_2.append(count)
    count+=1
percentage0_2 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions0_3.argmax(axis=1))
correct = 0
count = 0
wrong0_3 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_3.append(count)
    count+=1
percentage0_3 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions0_4.argmax(axis=1))
correct = 0
count = 0
wrong0_4 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_4.append(count)
    count+=1
percentage0_4 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions0_5.argmax(axis=1))
correct = 0
count = 0
wrong0_5 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_5.append(count)
    count+=1
percentage0_5 = correct/len(open_test_data['label'])

print(percentage0_1, percentage0_2, percentage0_3, percentage0_4, percentage0_5)

print('wrong0_1', wrong0_1)
print('wrong0_2', wrong0_2)
print('wrong0_3', wrong0_3)
print('wrong0_4', wrong0_4)
print('wrong0_5', wrong0_5)

0.6957494407158836 0.6923937360178971 0.6935123042505593 0.020879940343027592 0.6991051454138703
wrong0_1 [0, 2, 7, 13, 14, 19, 20, 21, 24, 25, 28, 29, 32, 33, 34, 35, 40, 42, 43, 44, 48, 50, 53, 54, 57, 59, 64, 69, 70, 71, 72, 74, 76, 79, 81, 86, 90, 93, 97, 98, 99, 100, 102, 104, 108, 116, 119, 122, 123, 130, 137, 139, 142, 144, 147, 148, 149, 151, 152, 154, 157, 158, 160, 168, 172, 173, 175, 179, 184, 186, 191, 193, 195, 199, 201, 202, 203, 205, 208, 218, 224, 227, 229, 230, 231, 236, 237, 240, 243, 245, 252, 257, 258, 259, 262, 263, 265, 271, 276, 277, 281, 284, 285, 287, 290, 292, 295, 297, 301, 306, 307, 308, 312, 313, 314, 315, 316, 325, 326, 328, 336, 338, 339, 340, 341, 342, 347, 349, 350, 362, 367, 372, 374, 379, 380, 391, 392, 393, 395, 396, 401, 404, 405, 420, 422, 426, 427, 432, 440, 447, 449, 454, 457, 459, 462, 465, 466, 467, 473, 474, 475, 476, 477, 479, 481, 484, 489, 495, 497, 498, 508, 511, 522, 523, 531, 532, 536, 538, 546, 547, 550, 552, 555, 558, 559, 560, 561, 56

In [417]:
zip_list = zip(open_test_data['label'].values, test_predictions1_0.argmax(axis=1))
correct = 0
count = 0
wrong1_0 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_0.append(count)
    count+=1
percentage1_0 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions1_2.argmax(axis=1))
correct = 0
count = 0
wrong1_2 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_2.append(count)
    count+=1
percentage1_2 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions1_3.argmax(axis=1))
correct = 0
count = 0
wrong1_3 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_3.append(count)
    count+=1
percentage1_3 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions1_4.argmax(axis=1))
correct = 0
count = 0
wrong1_4 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_4.append(count)
    count+=1
percentage1_4 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions1_5.argmax(axis=1))
correct = 0
count = 0
wrong1_5 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_5.append(count)
    count+=1
percentage1_5 = correct/len(open_test_data['label'])

print(percentage1_0, percentage1_2, percentage1_3, percentage1_4, percentage1_5)

print('wrong1_0',wrong1_0)
print('wrong1_2',wrong1_2)
print('wrong1_3',wrong1_3)
print('wrong1_4',wrong1_4)
print('wrong1_5',wrong1_5)

0.6950037285607755 0.6923937360178971 0.6935123042505593 0.020879940343027592 0.6991051454138703
wrong1_0 [0, 2, 7, 13, 14, 19, 20, 21, 24, 25, 28, 29, 32, 33, 34, 35, 40, 42, 43, 44, 48, 50, 53, 54, 57, 59, 64, 69, 70, 71, 72, 74, 76, 78, 79, 81, 86, 90, 93, 97, 98, 99, 100, 102, 104, 108, 116, 119, 122, 123, 130, 137, 139, 142, 144, 147, 148, 149, 151, 152, 154, 157, 158, 160, 168, 172, 173, 175, 179, 184, 186, 191, 193, 195, 199, 201, 202, 203, 205, 208, 218, 224, 227, 229, 230, 231, 236, 237, 240, 243, 245, 252, 257, 258, 259, 262, 263, 265, 271, 276, 277, 281, 284, 285, 287, 290, 292, 295, 297, 301, 306, 307, 308, 312, 313, 314, 315, 316, 325, 326, 328, 336, 338, 339, 340, 341, 342, 347, 349, 350, 362, 367, 372, 374, 379, 380, 391, 392, 393, 395, 396, 401, 404, 405, 420, 422, 426, 427, 432, 440, 447, 449, 454, 457, 459, 462, 465, 466, 467, 473, 474, 475, 476, 477, 479, 481, 484, 489, 495, 497, 498, 508, 511, 522, 523, 531, 532, 536, 538, 546, 547, 550, 552, 555, 558, 559, 560, 561

In [418]:
zip_list = zip(open_test_data['label'].values, test_predictions2_0.argmax(axis=1))
correct = 0
count = 0
wrong2_0 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_0.append(count)
    count+=1
percentage2_0 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions2_1.argmax(axis=1))
correct = 0
count = 0
wrong2_1 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_1.append(count)
    count+=1
percentage2_1 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions2_3.argmax(axis=1))
correct = 0
count = 0
wrong2_3 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_3.append(count)
    count+=1
percentage2_3 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions2_4.argmax(axis=1))
correct = 0
count = 0
wrong2_4 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_4.append(count)
    count+=1
percentage2_4 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions2_5.argmax(axis=1))
correct = 0
count = 0
wrong2_5 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_5.append(count)
    count+=1
percentage2_5 = correct/len(open_test_data['label'])

print(percentage2_0, percentage2_1, percentage2_3, percentage2_4, percentage2_5)

print('wrong2_0',wrong2_0)
print('wrong2_1',wrong2_1)
print('wrong2_3',wrong2_3)
print('wrong2_4',wrong2_4)
print('wrong2_5',wrong2_5)

0.6950037285607755 0.6957494407158836 0.6935123042505593 0.020879940343027592 0.6991051454138703
wrong2_0 [0, 2, 7, 13, 14, 19, 20, 21, 24, 25, 28, 29, 32, 33, 34, 35, 40, 42, 43, 44, 48, 50, 53, 54, 57, 59, 64, 69, 70, 71, 72, 74, 76, 78, 79, 81, 86, 90, 93, 97, 98, 99, 100, 102, 104, 108, 116, 119, 122, 123, 130, 137, 139, 142, 144, 147, 148, 149, 151, 152, 154, 157, 158, 160, 168, 172, 173, 175, 179, 184, 186, 191, 193, 195, 199, 201, 202, 203, 205, 208, 218, 224, 227, 229, 230, 231, 236, 237, 240, 243, 245, 252, 257, 258, 259, 262, 263, 265, 271, 276, 277, 281, 284, 285, 287, 290, 292, 295, 297, 301, 306, 307, 308, 312, 313, 314, 315, 316, 325, 326, 328, 336, 338, 339, 340, 341, 342, 347, 349, 350, 362, 367, 372, 374, 379, 380, 391, 392, 393, 395, 396, 401, 404, 405, 420, 422, 426, 427, 432, 440, 447, 449, 454, 457, 459, 462, 465, 466, 467, 473, 474, 475, 476, 477, 479, 481, 484, 489, 495, 497, 498, 508, 511, 522, 523, 531, 532, 536, 538, 546, 547, 550, 552, 555, 558, 559, 560, 561

In [419]:
zip_list = zip(open_test_data['label'].values, test_predictions3_0.argmax(axis=1))
correct = 0
count = 0
wrong3_0 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_0.append(count)
    count+=1
percentage3_0 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions3_2.argmax(axis=1))
correct = 0
count = 0
wrong3_2 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_2.append(count)
    count+=1
percentage3_2 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions3_1.argmax(axis=1))
correct = 0
count = 0
wrong3_1 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_1.append(count)
    count+=1
percentage3_1 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions3_4.argmax(axis=1))
correct = 0
count = 0
wrong3_4 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_4.append(count)
    count+=1
percentage3_4 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions3_5.argmax(axis=1))
correct = 0
count = 0
wrong3_5 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_5.append(count)
    count+=1
percentage3_5 = correct/len(open_test_data['label'])

print(percentage3_0, percentage3_2, percentage3_1, percentage3_4, percentage3_5)

print('wrong3_0',wrong3_0)
print('wrong3_2',wrong3_2)
print('wrong3_1',wrong3_1)
print('wrong3_4',wrong3_4)
print('wrong3_5',wrong3_5)

0.6950037285607755 0.6923937360178971 0.6957494407158836 0.020879940343027592 0.6991051454138703
wrong3_0 [0, 2, 7, 13, 14, 19, 20, 21, 24, 25, 28, 29, 32, 33, 34, 35, 40, 42, 43, 44, 48, 50, 53, 54, 57, 59, 64, 69, 70, 71, 72, 74, 76, 78, 79, 81, 86, 90, 93, 97, 98, 99, 100, 102, 104, 108, 116, 119, 122, 123, 130, 137, 139, 142, 144, 147, 148, 149, 151, 152, 154, 157, 158, 160, 168, 172, 173, 175, 179, 184, 186, 191, 193, 195, 199, 201, 202, 203, 205, 208, 218, 224, 227, 229, 230, 231, 236, 237, 240, 243, 245, 252, 257, 258, 259, 262, 263, 265, 271, 276, 277, 281, 284, 285, 287, 290, 292, 295, 297, 301, 306, 307, 308, 312, 313, 314, 315, 316, 325, 326, 328, 336, 338, 339, 340, 341, 342, 347, 349, 350, 362, 367, 372, 374, 379, 380, 391, 392, 393, 395, 396, 401, 404, 405, 420, 422, 426, 427, 432, 440, 447, 449, 454, 457, 459, 462, 465, 466, 467, 473, 474, 475, 476, 477, 479, 481, 484, 489, 495, 497, 498, 508, 511, 522, 523, 531, 532, 536, 538, 546, 547, 550, 552, 555, 558, 559, 560, 561

In [420]:
zip_list = zip(open_test_data['label'].values, test_predictions4_0.argmax(axis=1))
correct = 0
count = 0
wrong4_0 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_0.append(count)
    count+=1
percentage4_0 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions4_2.argmax(axis=1))
correct = 0
count = 0
wrong4_2 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_2.append(count)
    count+=1
percentage4_2 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions4_3.argmax(axis=1))
correct = 0
count = 0
wrong4_3 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_3.append(count)
    count+=1
percentage4_3 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions4_1.argmax(axis=1))
correct = 0
count = 0
wrong4_1 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_1.append(count)
    count+=1
percentage4_1 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions4_5.argmax(axis=1))
correct = 0
count = 0
wrong4_5 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_5.append(count)
    count+=1
percentage4_5 = correct/len(open_test_data['label'])

print(percentage4_0, percentage4_2, percentage4_3, percentage4_1, percentage4_5)

print('wrong4_0',wrong4_0)
print('wrong4_2',wrong4_2)
print('wrong4_3',wrong4_3)
print('wrong4_1',wrong4_1)
print('wrong4_5',wrong4_5)

0.6950037285607755 0.6923937360178971 0.6935123042505593 0.6957494407158836 0.6991051454138703
wrong4_0 [0, 2, 7, 13, 14, 19, 20, 21, 24, 25, 28, 29, 32, 33, 34, 35, 40, 42, 43, 44, 48, 50, 53, 54, 57, 59, 64, 69, 70, 71, 72, 74, 76, 78, 79, 81, 86, 90, 93, 97, 98, 99, 100, 102, 104, 108, 116, 119, 122, 123, 130, 137, 139, 142, 144, 147, 148, 149, 151, 152, 154, 157, 158, 160, 168, 172, 173, 175, 179, 184, 186, 191, 193, 195, 199, 201, 202, 203, 205, 208, 218, 224, 227, 229, 230, 231, 236, 237, 240, 243, 245, 252, 257, 258, 259, 262, 263, 265, 271, 276, 277, 281, 284, 285, 287, 290, 292, 295, 297, 301, 306, 307, 308, 312, 313, 314, 315, 316, 325, 326, 328, 336, 338, 339, 340, 341, 342, 347, 349, 350, 362, 367, 372, 374, 379, 380, 391, 392, 393, 395, 396, 401, 404, 405, 420, 422, 426, 427, 432, 440, 447, 449, 454, 457, 459, 462, 465, 466, 467, 473, 474, 475, 476, 477, 479, 481, 484, 489, 495, 497, 498, 508, 511, 522, 523, 531, 532, 536, 538, 546, 547, 550, 552, 555, 558, 559, 560, 561, 

In [421]:
zip_list = zip(open_test_data['label'].values, test_predictions5_0.argmax(axis=1))
correct = 0
count = 0
wrong5_0 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_0.append(count)
    count+=1
percentage5_0 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions5_2.argmax(axis=1))
correct = 0
count = 0
wrong5_2 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_2.append(count)
    count+=1
percentage5_2 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions5_3.argmax(axis=1))
correct = 0
count = 0
wrong5_3 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_3.append(count)
    count+=1
percentage5_3 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions5_1.argmax(axis=1))
correct = 0
count = 0
wrong5_1 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_1.append(count)
    count+=1
percentage5_1 = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions5_4.argmax(axis=1))
correct = 0
count = 0
wrong5_4 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_4.append(count)
    count+=1
percentage5_4 = correct/len(open_test_data['label'])

print(percentage5_0, percentage5_2, percentage5_3, percentage5_1, percentage5_4)

print('wrong5_0',wrong5_0)
print('wrong5_2',wrong5_2)
print('wrong5_3',wrong5_3)
print('wrong5_1',wrong5_1)
print('wrong5_4',wrong5_4)

0.6950037285607755 0.6923937360178971 0.6935123042505593 0.6957494407158836 0.020879940343027592
wrong5_0 [0, 2, 7, 13, 14, 19, 20, 21, 24, 25, 28, 29, 32, 33, 34, 35, 40, 42, 43, 44, 48, 50, 53, 54, 57, 59, 64, 69, 70, 71, 72, 74, 76, 78, 79, 81, 86, 90, 93, 97, 98, 99, 100, 102, 104, 108, 116, 119, 122, 123, 130, 137, 139, 142, 144, 147, 148, 149, 151, 152, 154, 157, 158, 160, 168, 172, 173, 175, 179, 184, 186, 191, 193, 195, 199, 201, 202, 203, 205, 208, 218, 224, 227, 229, 230, 231, 236, 237, 240, 243, 245, 252, 257, 258, 259, 262, 263, 265, 271, 276, 277, 281, 284, 285, 287, 290, 292, 295, 297, 301, 306, 307, 308, 312, 313, 314, 315, 316, 325, 326, 328, 336, 338, 339, 340, 341, 342, 347, 349, 350, 362, 367, 372, 374, 379, 380, 391, 392, 393, 395, 396, 401, 404, 405, 420, 422, 426, 427, 432, 440, 447, 449, 454, 457, 459, 462, 465, 466, 467, 473, 474, 475, 476, 477, 479, 481, 484, 489, 495, 497, 498, 508, 511, 522, 523, 531, 532, 536, 538, 546, 547, 550, 552, 555, 558, 559, 560, 561

In [156]:
list1 = sorted(test_predictions1.max(axis=1))

In [222]:
np.argsort(test_predictions1[893].max(), axis=0)

array([0], dtype=int64)

In [226]:
test_predictions1[893]

array([2.6349176e-03, 4.7732633e-01, 1.3522503e-03, 6.6169369e-04,
       3.9443374e-04, 3.1255971e-04, 4.9038912e-04, 5.4255122e-04,
       3.5288493e-04, 4.0631549e-04, 3.4434008e-04, 4.5492311e-04,
       7.3857157e-04, 4.3799143e-04, 8.4465288e-04, 1.4496159e-03,
       3.2529363e-04, 4.9278809e-04, 1.6025386e-03, 3.7133691e-04,
       4.9637303e-01, 4.6180759e-04, 5.1987678e-04, 3.1055711e-04,
       2.8761374e-04, 7.5126847e-04, 1.7959154e-03, 5.8591680e-04,
       2.6981980e-03, 5.3701643e-04, 9.1108697e-04, 4.8458859e-04,
       3.4642126e-04, 8.7107660e-04, 3.6518785e-04, 8.1867183e-04,
       3.4534198e-04], dtype=float32)

In [225]:
test_predictions1[893].max(axis=0)

0.49637303

In [227]:
test_predictions1[893].argmax(axis=0)

20

In [234]:
np.argsort(test_predictions1[893])[-2]

1

In [235]:
np.argsort(test_predictions1[1368])[-2]

20

In [260]:
test_predictions5[1368][20]

0.44388986

In [267]:
test_predictions0_5[1368][20]

0.44388986

In [451]:
test_predictions0123 = test_predictions0 + test_predictions1 + test_predictions2 + test_predictions3

In [310]:
test_predictions123

array([[2.25668133e-04, 8.39276181e-05, 1.20752797e-04, ...,
        8.14664061e-04, 1.20222729e-04, 1.50332402e-04],
       [4.67872422e-04, 2.79594446e-04, 4.14223410e-04, ...,
        4.04158287e-04, 3.98177071e-04, 1.67884631e-04],
       [3.65178421e-05, 1.07085500e-04, 3.57134777e-05, ...,
        3.17091253e-05, 3.90603345e-05, 1.55322759e-05],
       ...,
       [2.10242026e-04, 2.59420398e-04, 2.05724937e-04, ...,
        2.54305371e-04, 1.88049089e-04, 1.31045119e-04],
       [1.50147625e-05, 3.43265565e-05, 3.70246635e-05, ...,
        1.86712714e-05, 2.32220336e-05, 1.39054082e-05],
       [2.54165556e-04, 3.43393549e-05, 7.02933030e-05, ...,
        5.60420376e-05, 5.96607133e-05, 5.85412345e-05]], dtype=float32)

In [452]:
zip_list = zip(open_test_data['label'].values, test_predictions0123.argmax(axis=1))
correct = 0
count = 0
wrong0123 = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0123.append(count)
    count+=1
percentage0123 = correct/len(open_test_data['label'])

In [453]:
percentage0123

0.831841909023117

In [336]:
 test_predictions0123[1368][20]

2.2832367

In [337]:
 test_predictions0123[1368][1]

0.0004091191

In [339]:
print(np.argsort(test_predictions0123[1368])[-1], np.argsort(test_predictions0123[1368])[-2])

20 18


In [340]:
test_predictions0123[1368][20]

2.2832367

In [341]:
test_predictions0123[1368][18]

1.6433449

In [342]:
test_predictions0[1368][18]

0.08807404

In [321]:
test_predictions123[893].argmax()

20

In [422]:
test_predictions0_half = test_predictions0_1 + test_predictions0_2 + test_predictions0_3
test_predictions0_all = test_predictions0_1 + test_predictions0_2 + test_predictions0_3 + test_predictions0_4 + test_predictions0_5

In [423]:
test_predictions1_half = test_predictions1_0 + test_predictions1_2 + test_predictions1_3
test_predictions1_all = test_predictions1_0 + test_predictions1_2 + test_predictions1_3 + test_predictions1_4 + test_predictions1_5

In [424]:
test_predictions2_half = test_predictions2_1 + test_predictions2_0 + test_predictions2_3
test_predictions2_all = test_predictions2_1 + test_predictions2_0 + test_predictions2_3 + test_predictions2_4 + test_predictions2_5

In [425]:
test_predictions3_half = test_predictions3_1 + test_predictions3_2 + test_predictions3_0
test_predictions3_all = test_predictions3_1 + test_predictions3_2 + test_predictions3_0 + test_predictions3_4 + test_predictions3_5

In [426]:
test_predictions4_half = test_predictions4_0 + test_predictions4_1 + test_predictions4_2 + test_predictions4_3
test_predictions4_all = test_predictions4_0 + test_predictions4_1 + test_predictions4_2 + test_predictions4_3 + test_predictions4_5

In [427]:
test_predictions5_half = test_predictions5_0 + test_predictions5_1 + test_predictions5_2 + test_predictions5_3
test_predictions5_all = test_predictions5_0 + test_predictions5_1 + test_predictions5_2 + test_predictions5_3 + test_predictions5_4

In [428]:
test_predictionshalf_half = test_predictions0_half + test_predictions1_half + test_predictions2_half+ test_predictions3_half
test_predictionshalf_all = test_predictions0_all + test_predictions1_all + test_predictions2_all+ test_predictions3_all
test_predictionsall_half = test_predictionshalf_half + test_predictions4_half + test_predictions5_half
test_predictionsall_all = test_predictionshalf_all + test_predictions4_all + test_predictions5_all

In [429]:
zip_list = zip(open_test_data['label'].values, test_predictions0_half.argmax(axis=1))
correct = 0
count = 0
wrong0_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_half.append(count)
    count+=1
percentage0_half = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions1_half.argmax(axis=1))
correct = 0
count = 0
wrong1_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_half.append(count)
    count+=1
percentage1_half = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions2_half.argmax(axis=1))
correct = 0
count = 0
wrong2_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_half.append(count)
    count+=1
percentage2_half = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions3_half.argmax(axis=1))
correct = 0
count = 0
wrong3_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_half.append(count)
    count+=1
percentage3_half = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions4_half.argmax(axis=1))
correct = 0
count = 0
wrong4_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_half.append(count)
    count+=1
percentage4_half = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions5_half.argmax(axis=1))
correct = 0
count = 0
wrong5_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_half.append(count)
    count+=1
percentage5_half = correct/len(open_test_data['label'])

In [430]:
zip_list = zip(open_test_data['label'].values, test_predictions0_all.argmax(axis=1))
correct = 0
count = 0
wrong0_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong0_all.append(count)
    count+=1
percentage0_all = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions1_all.argmax(axis=1))
correct = 0
count = 0
wrong1_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong1_all.append(count)
    count+=1
percentage1_all = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions2_all.argmax(axis=1))
correct = 0
count = 0
wrong2_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong2_all.append(count)
    count+=1
percentage2_all = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions3_all.argmax(axis=1))
correct = 0
count = 0
wrong3_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong3_all.append(count)
    count+=1
percentage3_all = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions4_all.argmax(axis=1))
correct = 0
count = 0
wrong4_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong4_all.append(count)
    count+=1
percentage4_all = correct/len(open_test_data['label'])

zip_list = zip(open_test_data['label'].values, test_predictions5_all.argmax(axis=1))
correct = 0
count = 0
wrong5_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrong5_all.append(count)
    count+=1
percentage5_all = correct/len(open_test_data['label'])

In [431]:
zip_list = zip(open_test_data['label'].values, test_predictionshalf_half.argmax(axis=1))
correct = 0
count = 0
wronghalf_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wronghalf_half.append(count)
    count+=1
percentagehalf_half = correct/len(open_test_data['label'])

In [432]:
zip_list = zip(open_test_data['label'].values, test_predictionshalf_all.argmax(axis=1))
correct = 0
count = 0
wronghalf_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wronghalf_all.append(count)
    count+=1
percentagehalf_all = correct/len(open_test_data['label'])

In [433]:
zip_list = zip(open_test_data['label'].values, test_predictionsall_half.argmax(axis=1))
correct = 0
count = 0
wrongall_half = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrongall_half.append(count)
    count+=1
percentageall_half = correct/len(open_test_data['label'])

In [434]:
zip_list = zip(open_test_data['label'].values, test_predictionsall_all.argmax(axis=1))
correct = 0
count = 0
wrongall_all = []
for ans, pred in zip_list:
    if(ans==pred):
        correct += 1
    else:
        wrongall_all.append(count)
    count+=1
percentageall_all = correct/len(open_test_data['label'])

In [435]:
print(percentage0_half,
     percentage1_half,
     percentage2_half,
     percentage3_half,
     percentage4_half,
     percentage5_half)

print(percentage0_all,
     percentage1_all,
     percentage2_all,
     percentage3_all,
     percentage4_all,
     percentage5_all)

print(percentagehalf_half,
     percentagehalf_all,
     percentageall_half,
     percentageall_all,)

0.6953765846383296 0.6946308724832215 0.6953765846383296 0.6957494407158836 0.6957494407158836 0.6957494407158836
0.6968680089485458 0.6968680089485458 0.6968680089485458 0.6972408650260999 0.6964951528709918 0.6964951528709918
0.6957494407158836 0.6961222967934377 0.6957494407158836 0.6972408650260999


In [436]:
print("half")
print(np.argsort(test_predictions0_half[893])[-1], np.argsort(test_predictions0_half[893])[-2])
print(np.argsort(test_predictions1_half[893])[-1], np.argsort(test_predictions1_half[893])[-2])
print(np.argsort(test_predictions2_half[893])[-1], np.argsort(test_predictions2_half[893])[-2])
print(np.argsort(test_predictions3_half[893])[-1], np.argsort(test_predictions3_half[893])[-2])
print(np.argsort(test_predictions4_half[893])[-1], np.argsort(test_predictions4_half[893])[-2])
print(np.argsort(test_predictions5_half[893])[-1], np.argsort(test_predictions5_half[893])[-2])
print("all")
print(np.argsort(test_predictions0_all[893])[-1], np.argsort(test_predictions0_all[893])[-2])
print(np.argsort(test_predictions1_all[893])[-1], np.argsort(test_predictions1_all[893])[-2])
print(np.argsort(test_predictions2_all[893])[-1], np.argsort(test_predictions2_all[893])[-2])
print(np.argsort(test_predictions3_all[893])[-1], np.argsort(test_predictions3_all[893])[-2])
print(np.argsort(test_predictions4_all[893])[-1], np.argsort(test_predictions4_all[893])[-2])
print(np.argsort(test_predictions5_all[893])[-1], np.argsort(test_predictions5_all[893])[-2])

half
1 18
1 18
1 18
1 18
1 18
1 18
all
1 35
1 35
1 35
1 35
1 18
1 35


In [437]:
print("half")
print(np.argsort(test_predictions0_half[1368])[-1], np.argsort(test_predictions0_half[1368])[-2])
print(np.argsort(test_predictions1_half[1368])[-1], np.argsort(test_predictions1_half[1368])[-2])
print(np.argsort(test_predictions2_half[1368])[-1], np.argsort(test_predictions2_half[1368])[-2])
print(np.argsort(test_predictions3_half[1368])[-1], np.argsort(test_predictions3_half[1368])[-2])
print(np.argsort(test_predictions4_half[1368])[-1], np.argsort(test_predictions4_half[1368])[-2])
print(np.argsort(test_predictions5_half[1368])[-1], np.argsort(test_predictions5_half[1368])[-2])
print("all")
print(np.argsort(test_predictions0_all[1368])[-1], np.argsort(test_predictions0_all[1368])[-2])
print(np.argsort(test_predictions1_all[1368])[-1], np.argsort(test_predictions1_all[1368])[-2])
print(np.argsort(test_predictions2_all[1368])[-1], np.argsort(test_predictions2_all[1368])[-2])
print(np.argsort(test_predictions3_all[1368])[-1], np.argsort(test_predictions3_all[1368])[-2])
print(np.argsort(test_predictions4_all[1368])[-1], np.argsort(test_predictions4_all[1368])[-2])
print(np.argsort(test_predictions5_all[1368])[-1], np.argsort(test_predictions5_all[1368])[-2])

half
31 18
31 18
31 18
31 18
31 18
31 18
all
31 35
31 35
31 35
35 31
31 18
35 31


In [438]:
print(np.argsort(test_predictionshalf_half[893])[-1], np.argsort(test_predictionshalf_half[893])[-2])
print(np.argsort(test_predictionshalf_all[893])[-1], np.argsort(test_predictionshalf_all[893])[-2])
print(np.argsort(test_predictionsall_half[893])[-1], np.argsort(test_predictionsall_half[893])[-2])
print(np.argsort(test_predictionsall_all[893])[-1], np.argsort(test_predictionsall_all[893])[-2])

1 18
1 35
1 18
1 35


In [439]:
print(np.argsort(test_predictionshalf_half[1368])[-1], np.argsort(test_predictionshalf_half[1368])[-2])
print(np.argsort(test_predictionshalf_all[1368])[-1], np.argsort(test_predictionshalf_all[1368])[-2])
print(np.argsort(test_predictionsall_half[1368])[-1], np.argsort(test_predictionsall_half[1368])[-2])
print(np.argsort(test_predictionsall_all[1368])[-1], np.argsort(test_predictionsall_all[1368])[-2])

31 18
31 35
31 18
31 35


In [440]:
print(np.argsort(test_predictions0_half[1368])[-1], np.argsort(test_predictions0_half[1368])[-2])
print(np.argsort(test_predictions1_half[1368])[-1], np.argsort(test_predictions1_half[1368])[-2])
print(np.argsort(test_predictions2_half[1368])[-1], np.argsort(test_predictions2_half[1368])[-2])
print(np.argsort(test_predictions3_half[1368])[-1], np.argsort(test_predictions3_half[1368])[-2])
print(np.argsort(test_predictions4_half[1368])[-1], np.argsort(test_predictions4_half[1368])[-2])
print(np.argsort(test_predictions5_half[1368])[-1], np.argsort(test_predictions5_half[1368])[-2])

print(np.argsort(test_predictions0_all[1368])[-1], np.argsort(test_predictions0_all[1368])[-2])
print(np.argsort(test_predictions1_all[1368])[-1], np.argsort(test_predictions1_all[1368])[-2])
print(np.argsort(test_predictions2_all[1368])[-1], np.argsort(test_predictions2_all[1368])[-2])
print(np.argsort(test_predictions3_all[1368])[-1], np.argsort(test_predictions3_all[1368])[-2])
print(np.argsort(test_predictions4_all[1368])[-1], np.argsort(test_predictions4_all[1368])[-2])
print(np.argsort(test_predictions5_all[1368])[-1], np.argsort(test_predictions5_all[1368])[-2])

31 18
31 18
31 18
31 18
31 18
31 18
31 35
31 35
31 35
35 31
31 18
35 31


In [441]:
print(np.argsort(test_predictions0[893])[-1], np.argsort(test_predictions0[893])[-2])
print(np.argsort(test_predictions1[893])[-1], np.argsort(test_predictions1[893])[-2])
print(np.argsort(test_predictions2[893])[-1], np.argsort(test_predictions2[893])[-2])
print(np.argsort(test_predictions3[893])[-1], np.argsort(test_predictions3[893])[-2])
print(np.argsort(test_predictions4[893])[-1], np.argsort(test_predictions4[893])[-2])
print(np.argsort(test_predictions5[893])[-1], np.argsort(test_predictions5[893])[-2])

1 20
1 20
1 20
1 18
15 1
1 5


In [442]:
print(np.argsort(test_predictions0_1[893])[-1], np.argsort(test_predictions0_1[893])[-2])
print(np.argsort(test_predictions0_2[893])[-1], np.argsort(test_predictions0_2[893])[-2])
print(np.argsort(test_predictions0_3[893])[-1], np.argsort(test_predictions0_3[893])[-2])
print(np.argsort(test_predictions0_4[893])[-1], np.argsort(test_predictions0_4[893])[-2])
print(np.argsort(test_predictions0_5[893])[-1], np.argsort(test_predictions0_5[893])[-2])

1 5
1 18
1 18
35 26
1 5


In [443]:
print(np.argsort(test_predictions1_0[893])[-1], np.argsort(test_predictions1_0[893])[-2])
print(np.argsort(test_predictions1_2[893])[-1], np.argsort(test_predictions1_2[893])[-2])
print(np.argsort(test_predictions1_3[893])[-1], np.argsort(test_predictions1_3[893])[-2])
print(np.argsort(test_predictions1_4[893])[-1], np.argsort(test_predictions1_4[893])[-2])
print(np.argsort(test_predictions1_5[893])[-1], np.argsort(test_predictions1_5[893])[-2])

1 5
1 18
1 18
35 26
1 5


In [444]:
print(np.argsort(test_predictions2_0[893])[-1], np.argsort(test_predictions2_0[893])[-2])
print(np.argsort(test_predictions2_1[893])[-1], np.argsort(test_predictions2_1[893])[-2])
print(np.argsort(test_predictions2_3[893])[-1], np.argsort(test_predictions2_3[893])[-2])
print(np.argsort(test_predictions2_4[893])[-1], np.argsort(test_predictions2_4[893])[-2])
print(np.argsort(test_predictions2_5[893])[-1], np.argsort(test_predictions2_5[893])[-2])

1 5
1 5
1 18
35 26
1 5


In [445]:
print(np.argsort(test_predictions3_0[893])[-1], np.argsort(test_predictions3_0[893])[-2])
print(np.argsort(test_predictions3_1[893])[-1], np.argsort(test_predictions3_1[893])[-2])
print(np.argsort(test_predictions3_2[893])[-1], np.argsort(test_predictions3_2[893])[-2])
print(np.argsort(test_predictions3_4[893])[-1], np.argsort(test_predictions3_4[893])[-2])
print(np.argsort(test_predictions3_5[893])[-1], np.argsort(test_predictions3_5[893])[-2])

1 5
1 5
1 18
35 26
1 5


In [446]:
print(np.argsort(test_predictions4_0[893])[-1], np.argsort(test_predictions4_0[893])[-2])
print(np.argsort(test_predictions4_1[893])[-1], np.argsort(test_predictions4_1[893])[-2])
print(np.argsort(test_predictions4_2[893])[-1], np.argsort(test_predictions4_2[893])[-2])
print(np.argsort(test_predictions4_3[893])[-1], np.argsort(test_predictions4_3[893])[-2])
print(np.argsort(test_predictions4_5[893])[-1], np.argsort(test_predictions4_5[893])[-2])

1 5
1 5
1 18
1 18
1 5


In [447]:
print(np.argsort(test_predictions5_0[893])[-1], np.argsort(test_predictions5_0[893])[-2])
print(np.argsort(test_predictions5_1[893])[-1], np.argsort(test_predictions5_1[893])[-2])
print(np.argsort(test_predictions5_2[893])[-1], np.argsort(test_predictions5_2[893])[-2])
print(np.argsort(test_predictions5_3[893])[-1], np.argsort(test_predictions5_3[893])[-2])
print(np.argsort(test_predictions5_4[893])[-1], np.argsort(test_predictions5_4[893])[-2])

1 5
1 5
1 18
1 18
35 26


In [261]:
np.max(test_predictions0_1[1368])

0.5337115

In [193]:
sort_test_pred1 = sorted(test_predictions1.max(axis=1),reverse=False)
count = 0
for i in sort_test_pred1:
    if i < 0.50:
        count += 1
print(count)

2


In [192]:
sort_test_pred2 = sorted(test_predictions2.max(axis=1),reverse=False)
count = 0
for i in sort_test_pred2:
    if i < 0.50:
        count += 1
print(count)

1


In [191]:
sort_test_pred3 = sorted(test_predictions3.max(axis=1),reverse=False)
count = 0
for i in sort_test_pred3:
    if i < 0.50:
        count += 1
print(count)

2


In [190]:
sort_test_pred4 = sorted(test_predictions4.max(axis=1),reverse=False)
count = 0
for i in sort_test_pred4:
    if i < 0.50:
        count += 1
print(count)

274


In [184]:
sort_test_pred5 = sorted(test_predictions5.max(axis=1),reverse=False)
count = 0
for i in sort_test_pred5:
    if i < 0.50:
        count += 1
print(count)

114


In [208]:
test_predictions3_[893].max()

0.6879749

In [133]:
test_predictions3[893].max()

0.496781

In [136]:
test_predictions3_2[893].max()

0.6879749

In [28]:
test_predictions1.argmax(axis=1)

array([33,  6, 17, ..., 22, 18, 34], dtype=int64)

In [16]:
test_predictions2

array([[6.2358013e-04, 2.1808062e-04, 6.1475165e-04, ..., 5.1126839e-04,
        8.7359035e-04, 4.6595727e-05],
       [2.2786936e-04, 2.9028361e-05, 2.3115735e-04, ..., 3.8274706e-05,
        5.8605077e-05, 3.3263092e-05],
       [5.7085003e-06, 5.3871722e-06, 1.1827373e-05, ..., 5.2252522e-06,
        3.0896023e-05, 2.8574039e-05],
       ...,
       [1.0517626e-05, 9.3944518e-06, 4.7778831e-06, ..., 2.9509340e-05,
        1.1692599e-05, 7.5778735e-06],
       [4.2963130e-04, 1.1960221e-04, 3.0312661e-04, ..., 1.0786421e-04,
        1.9948088e-04, 1.6578412e-04],
       [1.3106690e-05, 2.3793637e-05, 2.2182845e-05, ..., 9.9480361e-01,
        4.1309441e-04, 8.5840194e-04]], dtype=float32)

In [17]:
test_predictions3

array([[4.76189714e-04, 1.79339127e-04, 1.85228462e-04, ...,
        4.55330126e-04, 9.11451760e-04, 6.14041492e-05],
       [1.43575075e-04, 3.51108465e-05, 1.58227485e-04, ...,
        6.59663201e-05, 9.39462625e-05, 4.18528398e-05],
       [6.71874113e-06, 7.57777980e-06, 1.35951732e-05, ...,
        5.83190740e-06, 2.57950942e-05, 2.95722202e-05],
       ...,
       [1.21163785e-05, 1.12831995e-05, 7.05116236e-06, ...,
        2.74903196e-05, 1.12880316e-05, 8.92948265e-06],
       [6.95524737e-04, 1.96239576e-04, 2.97457358e-04, ...,
        1.28116371e-04, 2.28530465e-04, 1.88074409e-04],
       [9.76733736e-06, 2.22070194e-05, 2.91578835e-05, ...,
        9.95537281e-01, 5.88897849e-04, 5.63493813e-04]], dtype=float32)

In [22]:
test_predictions4

array([[6.0483394e-03, 6.2603150e-03, 4.3824073e-03, ..., 2.6178598e-02,
        3.7694265e-04, 3.5494269e-04],
       [4.5502730e-04, 2.3380038e-04, 6.6620111e-04, ..., 1.9540139e-04,
        2.0825190e-03, 1.1469890e-04],
       [1.0445520e-05, 1.7451199e-05, 1.8588364e-05, ..., 6.1682586e-05,
        7.4562937e-05, 7.1665585e-05],
       ...,
       [1.4557211e-03, 2.2534319e-04, 4.1004596e-04, ..., 4.4426080e-03,
        3.4214228e-03, 3.2265735e-04],
       [7.8368007e-04, 9.3790155e-04, 1.4345295e-03, ..., 6.0565089e-04,
        8.9805711e-05, 1.9425007e-04],
       [1.6567766e-04, 9.4562660e-05, 2.6193185e-04, ..., 9.3274170e-01,
        8.0608977e-03, 9.2328442e-03]], dtype=float32)

In [23]:
test_predictions5

array([[3.64680745e-04, 1.74839457e-04, 3.84119339e-04, ...,
        5.05125558e-04, 6.42877771e-04, 3.22505730e-05],
       [1.47090090e-04, 3.67215653e-05, 1.85812707e-04, ...,
        6.89196168e-05, 1.44327118e-04, 2.62434187e-05],
       [3.24026587e-05, 2.39588262e-05, 2.01101957e-05, ...,
        1.81391351e-05, 1.01922888e-04, 3.88188309e-05],
       ...,
       [1.25371289e-05, 7.36366383e-06, 5.00282613e-06, ...,
        3.00996271e-05, 1.22082365e-05, 8.14425221e-06],
       [3.45522683e-04, 1.58135561e-04, 1.99305301e-04, ...,
        1.32219342e-04, 5.13754901e-04, 4.79766109e-04],
       [1.29891268e-05, 3.11375516e-05, 2.99810363e-05, ...,
        9.94804680e-01, 6.16037229e-04, 1.05567579e-03]], dtype=float32)

In [376]:
test_predictions

array([[3.8842089e-04, 9.8658915e-05, 1.9220541e-04, ..., 1.1361293e-03,
        2.0893250e-04, 1.9492055e-03],
       [1.6855529e-03, 1.7572835e-04, 5.8907818e-04, ..., 3.0967992e-04,
        2.1141237e-03, 2.6890243e-04],
       [3.0358782e-04, 3.2406375e-03, 1.3832428e-03, ..., 3.7622737e-04,
        4.7626131e-04, 3.5005203e-04],
       ...,
       [1.6813530e-04, 1.3912636e-04, 2.1999129e-04, ..., 2.2457165e-04,
        7.7291136e-04, 3.1955555e-04],
       [7.6917221e-04, 3.3593684e-04, 1.9677414e-02, ..., 1.8990072e-04,
        7.0570689e-04, 2.5088130e-04],
       [4.9666845e-04, 2.7456594e-05, 1.0102015e-04, ..., 3.5992758e-05,
        1.2247005e-04, 6.1042992e-05]], dtype=float32)

In [14]:
FILEPATH = '../Mondeique/predictions2/'

np.save(FILEPATH+'XGB_'+'ne0',test_predictions0)
np.save(FILEPATH+'XGB_'+'ne1',test_predictions1)
np.save(FILEPATH+'XGB_'+'ne2',test_predictions2)
np.save(FILEPATH+'XGB_'+'ne3',test_predictions3)
np.save(FILEPATH+'XGB_'+'ne4',test_predictions4)
np.save(FILEPATH+'XGB_'+'ne5',test_predictions5)

np.save(FILEPATH+'XGB_'+'ne0m',test_predictions0m)
np.save(FILEPATH+'XGB_'+'ne1m',test_predictions1m)
np.save(FILEPATH+'XGB_'+'ne2m',test_predictions2m)
np.save(FILEPATH+'XGB_'+'ne3m',test_predictions3m)
np.save(FILEPATH+'XGB_'+'ne4m',test_predictions4m)
np.save(FILEPATH+'XGB_'+'ne5m',test_predictions5m)

In [456]:
np.save(FILEPATH+'XGB_'+'ne0_ne1',test_predictions0_1)
np.save(FILEPATH+'XGB_'+'ne0_ne2',test_predictions0_2)
np.save(FILEPATH+'XGB_'+'ne0_ne3',test_predictions0_3)
np.save(FILEPATH+'XGB_'+'ne0_ne4',test_predictions0_4)
np.save(FILEPATH+'XGB_'+'ne0_ne5',test_predictions0_5)

In [457]:
np.save(FILEPATH+'XGB_'+'ne1_ne0',test_predictions1_0)
np.save(FILEPATH+'XGB_'+'ne1_ne2',test_predictions1_2)
np.save(FILEPATH+'XGB_'+'ne1_ne3',test_predictions1_3)
np.save(FILEPATH+'XGB_'+'ne1_ne4',test_predictions1_4)
np.save(FILEPATH+'XGB_'+'ne1_ne5',test_predictions1_5)

In [458]:
np.save(FILEPATH+'XGB_'+'ne2_ne1',test_predictions2_1)
np.save(FILEPATH+'XGB_'+'ne2_ne0',test_predictions2_0)
np.save(FILEPATH+'XGB_'+'ne2_ne3',test_predictions2_3)
np.save(FILEPATH+'XGB_'+'ne2_ne4',test_predictions2_4)
np.save(FILEPATH+'XGB_'+'ne2_ne5',test_predictions2_5)

In [459]:
np.save(FILEPATH+'XGB_'+'ne3_ne1',test_predictions3_1)
np.save(FILEPATH+'XGB_'+'ne3_ne2',test_predictions3_2)
np.save(FILEPATH+'XGB_'+'ne3_ne0',test_predictions3_0)
np.save(FILEPATH+'XGB_'+'ne3_ne4',test_predictions3_4)
np.save(FILEPATH+'XGB_'+'ne3_ne5',test_predictions3_5)

In [460]:
np.save(FILEPATH+'XGB_'+'ne4_ne1',test_predictions4_1)
np.save(FILEPATH+'XGB_'+'ne4_ne2',test_predictions4_2)
np.save(FILEPATH+'XGB_'+'ne4_ne3',test_predictions4_3)
np.save(FILEPATH+'XGB_'+'ne4_ne0',test_predictions4_0)
np.save(FILEPATH+'XGB_'+'ne4_ne5',test_predictions4_5)

In [461]:
np.save(FILEPATH+'XGB_'+'ne5_ne1',test_predictions5_1)
np.save(FILEPATH+'XGB_'+'ne5_ne2',test_predictions5_2)
np.save(FILEPATH+'XGB_'+'ne5_ne3',test_predictions5_3)
np.save(FILEPATH+'XGB_'+'ne5_ne4',test_predictions5_4)
np.save(FILEPATH+'XGB_'+'ne5_ne0',test_predictions5_0)

In [ ]:
##############################################################################################

In [25]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

xgb_preds_2=[]



for dev_index, val_index in skf.split(train_cvec_char2.tocsc(), ytrain):
    dev_X, val_X = train_cvec_char2.tocsc()[dev_index], train_cvec_char2.tocsc()[val_index]
    dev_y, val_y = ytrain[dev_index], ytrain[val_index]
    watchlist = [(dev_X, dev_y), (val_X, val_y)]
    watchmetric = ['merror', 'mlogloss']
    
    clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                            subsample=0.8, nthread=10, learning_rate=0.1)
    
    clf.fit(dev_X, dev_y)
    val_predictions = clf.predict_proba(val_X)
    test_predictions = clf.predict_proba(test_cvec_char2.tocsc())
    
    print ("XGB ctv logloss: %0.3f, accuracy: %0.3f, f1_score: %0.3f" % (multiclass_logloss(val_y, val_predictions), 
                                                                         prediction_accuracy(val_predictions, val_y), 
                                                                         f1_score(val_y, val_predictions.argmax(axis=1), average='micro')))
    xgb_preds_2.append(test_predictions)

XGB ctv logloss: 0.435, accuracy: 0.881, f1_score: 0.881
XGB ctv logloss: 0.384, accuracy: 0.905, f1_score: 0.905
XGB ctv logloss: 0.466, accuracy: 0.883, f1_score: 0.883
XGB ctv logloss: 0.485, accuracy: 0.879, f1_score: 0.879
XGB ctv logloss: 0.460, accuracy: 0.869, f1_score: 0.869
XGB ctv logloss: 0.474, accuracy: 0.873, f1_score: 0.873
XGB ctv logloss: 0.370, accuracy: 0.901, f1_score: 0.901
XGB ctv logloss: 0.435, accuracy: 0.888, f1_score: 0.888
XGB ctv logloss: 0.466, accuracy: 0.871, f1_score: 0.871
XGB ctv logloss: 0.502, accuracy: 0.879, f1_score: 0.879


In [26]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

xgb_preds_2_1=[]



for dev_index, val_index in skf.split(train_cvec_char2_1.tocsc(), ytrain):
    dev_X, val_X = train_cvec_char2_1.tocsc()[dev_index], train_cvec_char2_1.tocsc()[val_index]
    dev_y, val_y = ytrain[dev_index], ytrain[val_index]
    watchlist = [(dev_X, dev_y), (val_X, val_y)]
    watchmetric = ['merror', 'mlogloss']
    
    clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                            subsample=0.8, nthread=10, learning_rate=0.1)
    
    clf.fit(dev_X, dev_y)
    val_predictions = clf.predict_proba(val_X)
    test_predictions = clf.predict_proba(test_cvec_char2_1.tocsc())
    
    print ("XGB ctv logloss: %0.3f, accuracy: %0.3f, f1_score: %0.3f" % (multiclass_logloss(val_y, val_predictions), 
                                                                         prediction_accuracy(val_predictions, val_y), 
                                                                         f1_score(val_y, val_predictions.argmax(axis=1), average='micro')))
    xgb_preds_2_1.append(test_predictions)

XGB ctv logloss: 0.409, accuracy: 0.888, f1_score: 0.888
XGB ctv logloss: 0.377, accuracy: 0.907, f1_score: 0.907
XGB ctv logloss: 0.472, accuracy: 0.877, f1_score: 0.877
XGB ctv logloss: 0.472, accuracy: 0.873, f1_score: 0.873
XGB ctv logloss: 0.460, accuracy: 0.879, f1_score: 0.879
XGB ctv logloss: 0.473, accuracy: 0.866, f1_score: 0.866
XGB ctv logloss: 0.351, accuracy: 0.894, f1_score: 0.894
XGB ctv logloss: 0.437, accuracy: 0.896, f1_score: 0.896
XGB ctv logloss: 0.476, accuracy: 0.856, f1_score: 0.856
XGB ctv logloss: 0.498, accuracy: 0.882, f1_score: 0.882


In [27]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

xgb_preds_2_2=[]



for dev_index, val_index in skf.split(train_cvec_char2_2.tocsc(), ytrain):
    dev_X, val_X = train_cvec_char2_2.tocsc()[dev_index], train_cvec_char2_2.tocsc()[val_index]
    dev_y, val_y = ytrain[dev_index], ytrain[val_index]
    watchlist = [(dev_X, dev_y), (val_X, val_y)]
    watchmetric = ['merror', 'mlogloss']
    
    clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                            subsample=0.8, nthread=10, learning_rate=0.1)
    
    clf.fit(dev_X, dev_y)
    val_predictions = clf.predict_proba(val_X)
    test_predictions = clf.predict_proba(test_cvec_char2_2.tocsc())
    
    print ("XGB ctv logloss: %0.3f, accuracy: %0.3f, f1_score: %0.3f" % (multiclass_logloss(val_y, val_predictions), 
                                                                         prediction_accuracy(val_predictions, val_y), 
                                                                         f1_score(val_y, val_predictions.argmax(axis=1), average='micro')))
    xgb_preds_2_2.append(test_predictions)

XGB ctv logloss: 0.411, accuracy: 0.894, f1_score: 0.894
XGB ctv logloss: 0.384, accuracy: 0.903, f1_score: 0.903
XGB ctv logloss: 0.465, accuracy: 0.875, f1_score: 0.875
XGB ctv logloss: 0.487, accuracy: 0.875, f1_score: 0.875
XGB ctv logloss: 0.469, accuracy: 0.877, f1_score: 0.877
XGB ctv logloss: 0.482, accuracy: 0.869, f1_score: 0.869
XGB ctv logloss: 0.365, accuracy: 0.892, f1_score: 0.892
XGB ctv logloss: 0.451, accuracy: 0.873, f1_score: 0.873
XGB ctv logloss: 0.467, accuracy: 0.858, f1_score: 0.858
XGB ctv logloss: 0.499, accuracy: 0.886, f1_score: 0.886


In [36]:
xgb_preds_2[2].max(axis=1)

array([0.5394178 , 0.4711591 , 0.60229176, ..., 0.9207871 , 0.9863323 ,
       0.9364443 ], dtype=float32)

In [ ]:
test_predictions

In [30]:
xgb_preds_2_2

[array([[3.31616262e-04, 3.75245290e-04, 2.97131628e-04, ...,
         4.41962969e-04, 1.92853069e-04, 3.96920339e-04],
        [6.57173753e-01, 1.61453476e-03, 1.25216365e-01, ...,
         2.05072830e-03, 2.17495509e-03, 8.08643934e-04],
        [9.85371880e-04, 3.33238468e-02, 3.57390777e-03, ...,
         9.70782654e-04, 4.49901709e-04, 5.41437184e-04],
        ...,
        [1.47345945e-05, 1.10768233e-05, 1.30480666e-05, ...,
         5.56525556e-05, 4.15815084e-05, 1.76199883e-05],
        [1.05559557e-05, 1.25924462e-05, 1.38724981e-05, ...,
         1.04757673e-05, 1.66592108e-05, 1.09325692e-05],
        [4.23284037e-05, 9.77870368e-05, 4.28256426e-05, ...,
         3.79988123e-05, 3.35154800e-05, 2.38424709e-05]], dtype=float32),
 array([[5.9826660e-04, 3.9518086e-04, 3.7778972e-04, ..., 9.3364046e-04,
         3.0966938e-04, 7.8373670e-04],
        [8.4158289e-01, 1.0108527e-03, 6.4529292e-02, ..., 1.6272322e-03,
         1.0021975e-03, 5.5430457e-04],
        [7.4209092e-04

In [46]:
enc_prediction = lbl_enc.inverse_transform((np.mean(xgb_preds_2, axis=0)+np.mean(xgb_preds_2_1, axis=0)+np.mean(xgb_preds_2_2, axis=0)).argmax(axis=1))

In [43]:
enc_prediction_2 = lbl_enc.inverse_transform(np.mean(xgb_preds_2, axis=0).argmax(axis=1))

In [44]:
enc_prediction_2_1 = lbl_enc.inverse_transform(np.mean(xgb_preds_2_1, axis=0).argmax(axis=1))

In [45]:
enc_prediction_2_2 = lbl_enc.inverse_transform(np.mean(xgb_preds_2_2, axis=0).argmax(axis=1))